In [1]:
import pandas as pd


# Creating Grouped CSVs for September (including motion)

In [2]:
#Note zip needs to be unzipped and place accordingly
#Load CSV Files
data = pd.read_csv('data/RawComfortData.typed.csv')
data_motion = pd.read_csv('data/RawMotionData.typed.csv')

#Create a DateTime format
data['DateTime'] = pd.to_datetime(data['Timestamp'])
data_motion['DateTime'] = pd.to_datetime(data_motion['Timestamp'])

data = data.sort_values(['DateTime'])
data_motion = data_motion.sort_values(['DateTime'])

#Get individual room names
rooms = data['SpaceName'].unique()



In [3]:
#Get entries from Sept 01 and join motion with other sensor data
data = data[(data['DateTime'] >= '2019-09-01')]
data_motion = data_motion[(data_motion['DateTime'] >= '2019-09-01')]
data_merged = pd.concat([data,data_motion])

In [4]:
for room in rooms:
    print(room)
    room_data = data_merged[data_merged['SpaceName']==room].sort_values(['DateTime'])
    #Group events that occurred recently together. Grouping by eventtimestamp could group incorrectly
    room_data['group'] = (room_data['DateTime'].diff() > pd.Timedelta(seconds=240)).cumsum() #
    room_data = room_data.reset_index().pivot_table(index='group', columns='DataType', values='Value')
    #Insert 0 or 1 based on whether there has been a change in motion
    room_data.loc[(room_data['Motion'] != 0)&(room_data['Motion'].notnull()), 'Motion'] = 1
    room_data['Motion'] = room_data['Motion'].fillna(method='ffill')
    room_data['Motion']= room_data['Motion'].fillna(0)
    #To save:
#     room_data.to_csv('data/rooms_motion_from09/{}.csv'.format(room), index=False)


MDB 003 Kapel 1
MDB 009 Instructiezaal 01
MDB 108 Invliegende staf
MDB 302 Collegezaal achterzijde
MDB 010 Instructiezaal 02 raamzijde (docent)
MDB 209 Instructiezaal 07
MDB 109 Instructiezaal 03
MDB 115 Instructiezaal 05
MDB 110 instructiezaal 04 gangzijde (docent)
MDB 302 Collegezaal voorzijde (docent)


In [5]:
#Room 302
room_data.head()

DataType,CarbonDioxide,Humidity,Light,Motion,Sound,Temperature
group,,,,,,
0,382.0,63.0,0.0,0.0,8.0,22.1
1,383.0,63.0,0.0,0.0,8.0,22.1
2,384.0,64.0,0.0,0.0,8.0,22.1
3,384.0,64.0,0.0,0.0,8.0,22.1
4,386.0,64.0,0.0,0.0,9.0,22.1
